## Мотивация: Мат. формулировка задачи ML

$(x_i,y_i)$ - обучающая выборка ($x_i$ - вектор, $y_i$ - число)  
$f(x)$ - классификатор/регрессор  
$L(y_{pred}, y_{real})$ - функция потерь  
#### Задача
$$\min_f\sum_iL(f(x_i),y_i)$$
#### Конкретная задача 
Лин.регрессия:  
<img src="./images/regres1.gif" alt="Drawing" style="height: 150px;"/>
$x_i$ - число  
$y_i$ - число  
$f(x) = \alpha x$  
$L(y_{pred}, y_{real}) = (y_{pred} - y_{real})^2$


$$\min_\alpha\sum_i(\alpha x_i - y_i)^2$$

Как решать?  
Как в школе - находим производную, приравниваем к нулю.
$$(\sum_i(\alpha x_i - y_i)^2)'_{alpha} = (\sum_i(\alpha^2 x_i^2 - 2\alpha_ix_iy_i + y_i^2))'_{alpha}= \sum_i(\alpha^2 x_i^2 - 2\alpha_ix_iy_i + y_i^2)'_{alpha} = \sum_i(\alpha^2 x_i^2 - 2\alpha_ix_iy_i + y_i^2)'_{alpha} = \sum_i(2\alpha x_i^2 - 2x_iy_i) = $$
$$ =  2\alpha\sum_i x_i^2 - 2\sum_i x_iy_i $$
$$2\alpha\sum_i x_i^2 - 2\sum_i x_iy_i = 0$$
$$\alpha = \frac{\sum_i x_iy_i}{\sum_i x_i^2}$$
Решили задачу, нашли идеальную $f(x)$

В нейроночках всё примерно так же. Тоже нужна производная (градиент).

Мораль:
1. Задача обучения - задача оптимизации
2. Для того, чтобы модель хоро